In [1]:
import pandas as pd
import re
import numpy as np
import html
import matplotlib as plt
pd.set_option('display.max_columns', None)

In [2]:
dohmh_df = pd.read_csv("./data_csvs/DOHMH_New_York_City_Restaurant_Inspection_Results_20231202.csv")
dohmh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209789 entries, 0 to 209788
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  209789 non-null  int64  
 1   DBA                    209165 non-null  object 
 2   BORO                   209789 non-null  object 
 3   BUILDING               209481 non-null  object 
 4   STREET                 209783 non-null  object 
 5   ZIPCODE                207093 non-null  float64
 6   PHONE                  209781 non-null  object 
 7   CUISINE DESCRIPTION    207365 non-null  object 
 8   INSPECTION DATE        209789 non-null  object 
 9   ACTION                 207365 non-null  object 
 10  VIOLATION CODE         206208 non-null  object 
 11  VIOLATION DESCRIPTION  206208 non-null  object 
 12  CRITICAL FLAG          209789 non-null  object 
 13  SCORE                  199793 non-null  float64
 14  GRADE                  102961 non-nu

In [3]:
dohmh_df = dohmh_df.drop_duplicates()
#Change Zipcode to remove the .0
dohmh_df['ZIPCODE'] = dohmh_df['ZIPCODE'].fillna(0).astype(int)
dohmh_df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50106271,SOUR MOUSE,Manhattan,110,DELANCEY STREET,10002,6462567220,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,MN27,NaN
1,50117820,MODELLO BAR - BARCLAYS CENTER,Brooklyn,620,ATLANTIC AVENUE,11217,9174174384,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN
2,50141790,NaN,Manhattan,30,EAST 20 STREET,10003,9176670036,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,MN13,NaN
3,50134862,FINE AND RAW CHOCOLATE,Brooklyn,70,SCOTT AVENUE,11237,6462440734,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,NaN
4,50127238,THE DELUXE PARTY KTV,Queens,3420,LINDEN PL,11354,9172957843,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,QN22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,BLUEBERRY,Brooklyn,1849,CONEY ISLAND AVENUE,11230,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,BK43,NaN
209785,41444866,PIES-N-THIGHS,Brooklyn,166,SOUTH 4 STREET,11211,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,BK73,NaN
209786,50123805,QK Restaurant Inc.,Queens,4626,KISSENA BLVD,11355,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,QN22,NaN
209787,50056510,GINBO'S HAMBURGER HOUSE,Bronx,118,EAST 170 STREET,10452,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,BX63,NaN


In [4]:
#4.Remove any HTML Encoding from the columns.
def decode_html_entities(column):
    return column.apply(lambda x: html.unescape(str(x)))

dohmh_df['DBA'] = decode_html_entities(dohmh_df['DBA'])
dohmh_df['STREET'] = decode_html_entities(dohmh_df['STREET'])
dohmh_df['ACTION'] = decode_html_entities(dohmh_df['ACTION'])
dohmh_df['VIOLATION DESCRIPTION'] = decode_html_entities(dohmh_df['VIOLATION DESCRIPTION'])

In [5]:
def capitalize_column(dataframe, column):
    dataframe[column]=dataframe[column].str.title()
    
    #Standardize Business Suffixes
    pattern_1 = r'\bInc\b(?!\.)'
    pattern_2 = r'\bLlc\b(?!\.)'
    pattern_3 = r'\bCorp\b(?!\.)'
    pattern_4 = r'\bIncorporated\b(?!\.)'
    dataframe[column] = dataframe[column].str.replace(pattern_1,'Inc.')
    dataframe[column] = dataframe[column].str.replace(pattern_2,'LLC')
    dataframe[column] = dataframe[column].str.replace(pattern_3,'Corp.')
    dataframe[column] = dataframe[column].str.replace(pattern_4,'Inc.')
    
    #Standardize Location Abbreviations
    pattern_st = r'\bSt$|\bSt\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_st,'Street')
    pattern_av = r'\bAv$|\bAv\.$|\bAve$|\bAve\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_av,'Avenue')
    pattern_ln = r'\bLn$|\bLn\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_ln,'Lane')
    pattern_ln = r'\bBlvd$|\bBlvd\.$'
    dataframe[column] = dataframe[column].str.replace(pattern_ln,'Boulevard')
    
    #Change words ending in "'S" to ending in "'s" or "'T" to ending in "'t"
    pattern = r"(\w+)'S\b"
    replacement = lambda match: match.group(1).capitalize() + "'s"
    dataframe[column] = dataframe[column].str.replace(pattern, replacement, regex=True)
    pattern = r"(\w+)'T\b"
    replacement = lambda match: match.group(1).capitalize() + "'t"
    dataframe[column] = dataframe[column].str.replace(pattern, replacement, regex=True)
    return dataframe

dohmh_df = capitalize_column(dohmh_df,'DBA')
dohmh_df = capitalize_column(dohmh_df,'STREET')
dohmh_df

C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\1605767838.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_1,'Inc.')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\1605767838.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_2,'LLC')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\1605767838.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_3,'Corp.')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\1605767838.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(pattern_4,'Inc.')
C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\1605767838.py

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50106271,Sour Mouse,Manhattan,110,Delancey Street,10002,6462567220,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,MN27,NaN
1,50117820,Modello Bar - Barclays Center,Brooklyn,620,Atlantic Avenue,11217,9174174384,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN
2,50141790,Nan,Manhattan,30,East 20 Street,10003,9176670036,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,MN13,NaN
3,50134862,Fine And Raw Chocolate,Brooklyn,70,Scott Avenue,11237,6462440734,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,NaN
4,50127238,The Deluxe Party Ktv,Queens,3420,Linden Pl,11354,9172957843,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,QN22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,Blueberry,Brooklyn,1849,Coney Island Avenue,11230,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,BK43,NaN
209785,41444866,Pies-N-Thighs,Brooklyn,166,South 4 Street,11211,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,BK73,NaN
209786,50123805,Qk Restaurant Inc.,Queens,4626,Kissena Boulevard,11355,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,QN22,NaN
209787,50056510,Ginbo's Hamburger House,Bronx,118,East 170 Street,10452,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,BX63,NaN


In [6]:
dohmh_df.dropna(subset=['DBA'], inplace=True)
dohmh_df.dropna(subset=['STREET'], inplace=True)
dohmh_df=dohmh_df[dohmh_df['BORO']!='0']
dohmh_df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50106271,Sour Mouse,Manhattan,110,Delancey Street,10002,6462567220,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,MN27,NaN
1,50117820,Modello Bar - Barclays Center,Brooklyn,620,Atlantic Avenue,11217,9174174384,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN
2,50141790,Nan,Manhattan,30,East 20 Street,10003,9176670036,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,MN13,NaN
3,50134862,Fine And Raw Chocolate,Brooklyn,70,Scott Avenue,11237,6462440734,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,NaN
4,50127238,The Deluxe Party Ktv,Queens,3420,Linden Pl,11354,9172957843,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,QN22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,Blueberry,Brooklyn,1849,Coney Island Avenue,11230,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,BK43,NaN
209785,41444866,Pies-N-Thighs,Brooklyn,166,South 4 Street,11211,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,BK73,NaN
209786,50123805,Qk Restaurant Inc.,Queens,4626,Kissena Boulevard,11355,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,QN22,NaN
209787,50056510,Ginbo's Hamburger House,Bronx,118,East 170 Street,10452,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,BX63,NaN


In [7]:
def make_ordinal(number_str):
    # Use regex to find standalone numbers
    matches = re.findall(r'\b\d+\b', str(number_str))
    
    # Apply ordinal conversion to standalone numbers
    for match in matches:
        number = int(match)
        if number % 100 in [11, 12, 13]:
            suffix = 'th'
        else:
            suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(number % 10, 'th')
        number_with_suffix = str(number) + suffix
        
        # Replace the standalone number in the original string
        number_str = re.sub(r'\b{}\b'.format(match), number_with_suffix, str(number_str))

    return number_str

# Make Ordinal Numbers from Words
def words_to_ordinal(address):
    word_to_number = {'First': '1st', 'Second': '2nd', 'Third': '3rd', 'Fourth': '4th', 'Fifth': '5th', 'Sixth': '6th' \
                     , 'Seventh': '7th', 'Eight': '8th', 'Ninth': '9th', 'Tenth': '10th'}
    
    # Use regex to find words in the address and replace them with ordinal numbers
    for word, ordinal in word_to_number.items():
        address = re.sub(r'\b{}\b'.format(word), ordinal, address)

    return address

# Convert Capital suffixes to small
def convert_suffixes(number_str):
    # Use regex to find words with numbers followed by 'Th', 'Rd', 'Nd', 'St'
    matches = re.finditer(r'(\d+)([TtRrNnSs][HhDdNnTt])\b', str(number_str))
    
    # Apply the conversion to matched words
    for match in matches:
        number, suffix = match.groups()
        number_str = number_str.replace(match.group(0), f"{number}{suffix.lower()}")
    
    return number_str

dohmh_df['STREET']=dohmh_df['STREET'].apply(make_ordinal)
dohmh_df['STREET']=dohmh_df['STREET'].apply(words_to_ordinal)
dohmh_df['STREET']=dohmh_df['STREET'].apply(convert_suffixes)
dohmh_df['DBA']=dohmh_df['DBA'].apply(convert_suffixes)
dohmh_df

C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\3247547374.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dohmh_df['STREET']=dohmh_df['STREET'].apply(make_ordinal)
C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\3247547374.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dohmh_df['STREET']=dohmh_df['STREET'].apply(words_to_ordinal)
C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\3247547374.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50106271,Sour Mouse,Manhattan,110,Delancey Street,10002,6462567220,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,MN27,NaN
1,50117820,Modello Bar - Barclays Center,Brooklyn,620,Atlantic Avenue,11217,9174174384,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN
2,50141790,Nan,Manhattan,30,East 20th Street,10003,9176670036,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,MN13,NaN
3,50134862,Fine And Raw Chocolate,Brooklyn,70,Scott Avenue,11237,6462440734,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,NaN
4,50127238,The Deluxe Party Ktv,Queens,3420,Linden Pl,11354,9172957843,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,QN22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,Blueberry,Brooklyn,1849,Coney Island Avenue,11230,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,BK43,NaN
209785,41444866,Pies-N-Thighs,Brooklyn,166,South 4th Street,11211,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,BK73,NaN
209786,50123805,Qk Restaurant Inc.,Queens,4626,Kissena Boulevard,11355,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,QN22,NaN
209787,50056510,Ginbo's Hamburger House,Bronx,118,East 170th Street,10452,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,BX63,NaN


In [8]:
dohmh_df['Business Address'] = dohmh_df['BUILDING'] + " "+ dohmh_df['STREET'] + ", " +dohmh_df['BORO'] + ", NY"
dohmh_df

C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\3077022051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dohmh_df['Business Address'] = dohmh_df['BUILDING'] + " "+ dohmh_df['STREET'] + ", " +dohmh_df['BORO'] + ", NY"


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1,Business Address
0,50106271,Sour Mouse,Manhattan,110,Delancey Street,10002,6462567220,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,MN27,NaN,"110 Delancey Street, Manhattan, NY"
1,50117820,Modello Bar - Barclays Center,Brooklyn,620,Atlantic Avenue,11217,9174174384,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN,"620 Atlantic Avenue, Brooklyn, NY"
2,50141790,Nan,Manhattan,30,East 20th Street,10003,9176670036,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,MN13,NaN,"30 East 20th Street, Manhattan, NY"
3,50134862,Fine And Raw Chocolate,Brooklyn,70,Scott Avenue,11237,6462440734,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,NaN,"70 Scott Avenue, Brooklyn, NY"
4,50127238,The Deluxe Party Ktv,Queens,3420,Linden Pl,11354,9172957843,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,QN22,NaN,"3420 Linden Pl, Queens, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,Blueberry,Brooklyn,1849,Coney Island Avenue,11230,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,BK43,NaN,"1849 Coney Island Avenue, Brooklyn, NY"
209785,41444866,Pies-N-Thighs,Brooklyn,166,South 4th Street,11211,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,BK73,NaN,"166 South 4th Street, Brooklyn, NY"
209786,50123805,Qk Restaurant Inc.,Queens,4626,Kissena Boulevard,11355,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,QN22,NaN,"4626 Kissena Boulevard, Queens, NY"
209787,50056510,Ginbo's Hamburger House,Bronx,118,East 170th Street,10452,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,BX63,NaN,"118 East 170th Street, Bronx, NY"


In [9]:
nta_df = pd.read_csv("nta_data.csv")
nta_df

,NTA,Areas
0,BX98,Rikers Island
1,SI99,park-cemetery-etc-Staten Island
2,QN99,park-cemetery-etc-Queens
3,QN98,Airport
4,MN50,Stuyvesant Town-Cooper Village
...,...,...
190,BK35,Stuyvesant Heights
191,BK81,Brownsville
192,BK82,East New York
193,MN03,Central Harlem North-Polo Grounds


In [10]:
result_dict = nta_df.set_index('NTA')['Areas'].to_dict()

In [11]:
result_dict

{'BX98': 'Rikers Island',
 'SI99': 'park-cemetery-etc-Staten Island',
 'QN99': 'park-cemetery-etc-Queens',
 'QN98': 'Airport',
 'MN50': 'Stuyvesant Town-Cooper Village',
 'BK09': 'Brooklyn Heights-Cobble Hill',
 'QN41': 'Fresh Meadows-Utopia',
 'BX99': 'park-cemetery-etc-Bronx',
 'BK99': 'park-cemetery-etc-Brooklyn',
 'QN45': 'Douglas Manor-Douglaston-Little Neck',
 'QN47': 'Ft. Totten-Bay Terrace-Clearview',
 'QN62': 'Queensboro Hill',
 'QN44': 'Glen Oaks-Floral Park-New Hyde Park',
 'BK46': 'Ocean Parkway South',
 'QN48': 'Auburndale',
 'BX22': 'North Riverdale-Fieldston-Riverdale',
 'QN42': 'Oakland Gardens',
 'QN43': 'Bellerose',
 'SI32': 'Rossville-Woodrow',
 'SI11': 'Charleston-Richmond Valley-Tottenville',
 'SI25': 'Oakwood-Oakwood Beach',
 'QN60': 'Kew Gardens',
 'BK23': 'West Brighton',
 'QN18': 'Rego Park',
 'BX10': 'Pelham Bay-Country Club-City Island',
 'SI48': 'Arden Heights',
 'SI07': 'Westerleigh',
 'MN40': 'Upper East Side-Carnegie Hill',
 'SI01': "Annadale-Huguenot-Pri

In [12]:
dohmh_df['NTA'] = dohmh_df['NTA'].map(result_dict)
dohmh_df

C:\Users\Prateek\AppData\Local\Temp\ipykernel_30636\3464650564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dohmh_df['NTA'] = dohmh_df['NTA'].map(result_dict)


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1,Business Address
0,50106271,Sour Mouse,Manhattan,110,Delancey Street,10002,6462567220,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,Chinatown,NaN,"110 Delancey Street, Manhattan, NY"
1,50117820,Modello Bar - Barclays Center,Brooklyn,620,Atlantic Avenue,11217,9174174384,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,Park Slope-Gowanus,NaN,"620 Atlantic Avenue, Brooklyn, NY"
2,50141790,Nan,Manhattan,30,East 20th Street,10003,9176670036,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,Hudson Yards-Chelsea-Flatiron-Union Square,NaN,"30 East 20th Street, Manhattan, NY"
3,50134862,Fine And Raw Chocolate,Brooklyn,70,Scott Avenue,11237,6462440734,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,East Williamsburg,NaN,"70 Scott Avenue, Brooklyn, NY"
4,50127238,The Deluxe Party Ktv,Queens,3420,Linden Pl,11354,9172957843,NaN,01/01/1900,nan,NaN,nan,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,Flushing,NaN,"3420 Linden Pl, Queens, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,Blueberry,Brooklyn,1849,Coney Island Avenue,11230,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,Midwood,NaN,"1849 Coney Island Avenue, Brooklyn, NY"
209785,41444866,Pies-N-Thighs,Brooklyn,166,South 4th Street,11211,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,North Side-South Side,NaN,"166 South 4th Street, Brooklyn, NY"
209786,50123805,Qk Restaurant Inc.,Queens,4626,Kissena Boulevard,11355,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,Flushing,NaN,"4626 Kissena Boulevard, Queens, NY"
209787,50056510,Ginbo's Hamburger House,Bronx,118,East 170th Street,10452,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,West Concourse,NaN,"118 East 170th Street, Bronx, NY"


In [15]:
dohmh_df['CAMIS'].value_counts()

40398688    65
41406895    59
50045647    56
40365904    52
50111296    49
            ..
50107304     1
50112471     1
50144218     1
50112921     1
50144246     1
Name: CAMIS, Length: 28431, dtype: int64

In [18]:
dohmh_df[dohmh_df['CAMIS']==40398688]

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1,Business Address
311,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,05/08/2018,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Critical,13.0,A,05/08/2018,12/01/2023,Cycle Inspection / Re-inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
2295,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,05/07/2019,Violations were cited in the following area(s).,10A,Toilet facility not maintained and provided wi...,Not Critical,13.0,A,05/07/2019,12/01/2023,Cycle Inspection / Re-inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
6185,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,05/22/2017,Establishment Closed by DOHMH. Violations were...,02B,Hot food item not held at or above 140º F.,Critical,40.0,NaN,NaN,12/01/2023,Cycle Inspection / Re-inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
8351,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,11/16/2018,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,12.0,A,11/16/2018,12/01/2023,Cycle Inspection / Re-inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
9460,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,10/11/2019,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,70.0,C,10/11/2019,12/01/2023,Cycle Inspection / Re-inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195026,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,03/11/2020,Violations were cited in the following area(s).,20F,Current letter grade sign not posted.,Not Critical,NaN,NaN,NaN,12/01/2023,Administrative Miscellaneous / Initial Inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
198272,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,03/11/2020,Establishment Closed by DOHMH. Violations were...,05F,Insufficient or no refrigerated or hot holding...,Critical,35.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
201394,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,04/23/2018,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,17.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.609929,-73.922282,318.0,46.0,69800.0,3326830.0,3.084700e+09,Georgetown-Marine Park-Bergen Beach-Mill Basin,NaN,"5100 Kings Plaza Shopping Ct, Brooklyn, NY"
209048,40398688,Master Wok,Brooklyn,5100,Kings Plaza Shopping Ct,11234,7182586943,Chinese,05/07/2019,Violations were cited in

In [13]:
dohmh_df.to_csv('cleaned_dohmh.csv', index=False)

In [14]:
dohmh_df[dohmh_df['DBA']=='Mi Casa Restaurant']

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1,Business Address
14420,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,02/01/2023,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,32.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
25856,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,01/20/2022,Violations were cited in the following area(s).,10H,Proper sanitization not provided for utensil w...,Not Critical,48.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
27579,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,02/01/2023,Violations were cited in the following area(s).,02G,Cold TCS food item held above 41 °F; smoked or...,Critical,32.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
31099,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,01/20/2022,Violations were cited in the following area(s).,02G,Cold food item held above 41º F (smoked fish a...,Critical,48.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
35037,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,09/06/2023,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,61.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
36912,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,01/20/2022,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,48.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
37836,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,01/20/2022,Violations were cited in the following area(s).,10E,Accurate thermometer not provided in refrigera...,Not Critical,48.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
40248,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,01/20/2022,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,48.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
45843,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,10/04/2023,Violations were cited in the following area(s).,10B,Anti-siphonage or back-flow prevention device ...,Not Critical,13.0,A,10/04/2023,12/01/2023,Cycle Inspection / Re-inspection,40.699204,-73.833095,409.0,29.0,12200.0,4195822.0,4.093260e+09,Richmond Hill,NaN,"11620 Jamaica Avenue, Queens, NY"
48800,41658324,Mi Casa Restaurant,Queens,11620,Jamaica Avenue,11418,7188499636,Latin American,02/01/2023,Violations were cited in the